In [ ]:
PRETRAINED_MODELS_DIR = os.path.join("/content", "pretrained")
EFFICIENTDET_PYTORCH_DIR = os.path.join("/content", "efficientdet-pytorch")
EFFICIENTDET_PYTORCH_GIT_LINK = "https://github.com/rwightman/efficientdet-pytorch.git"

In [ ]:
import sys, os, subprocess
if not all(x in sys.path for x in [os.path.abspath(os.pardir), EFFICIENTDET_PYTORCH_DIR]):
    sys.path.insert(0, os.path.abspath(os.pardir))
    sys.path.insert(0, EFFICIENTDET_PYTORCH_DIR)
import CONFIG

%reload_ext autoreload
%autoreload 2

In [ ]:
def init_efficientdet():
    if not all([os.path.exists(PRETRAINED_MODELS_DIR), os.path.exists(EFFICIENTDET_PYTORCH_DIR)]):
        os.makedirs(PRETRAINED_MODELS_DIR, exist_ok=True)
        subprocess.call(f"git clone {EFFICIENTDET_PYTORCH_GIT_LINK}", shell=True, cwd=CONFIG.CFG.BASEPATH)

In [ ]:
init_efficientdet()

In [ ]:
import sys
import torch
import os
from datetime import datetime
import time
import random
import gc
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold, KFold
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [ ]:
DATA_DIR = CONFIG.CFG.DATA.BASE
MODELS_OUT_DIR = CONFIG.CFG.DATA.MODELS_OUT

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
ON_CPU = DEVICE == torch.device('cpu')
TRAIN_BATCH_SIZE = 4 if ON_CPU else 5
VALID_BATCH_SIZE = 4 if ON_CPU else 6
NUM_EPOCHS = 5 if ON_CPU else 10
LEARNING_RATE = 4e-5
NEW_COLUMNS = ['x', 'y', 'w', 'h']
IMAGE_SIZE = 512

In [ ]:
train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test_df = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))

UNIQ_TRAIN_IMAGE_IDS = train_df["image_id"].unique()

train_df.shape, test_df.shape

In [ ]:
# separate the bboxes into new columns
sep_bboxes = np.stack(train_df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, new_column in enumerate(NEW_COLUMNS):
    train_df[new_column] = sep_bboxes[:, i]
train_df.drop(columns=['bbox'], inplace=True)
train_df.head()

In [ ]:
train_df.loc[:, "fold"] = -1
kf = KFold(shuffle=True, random_state=42)
for fold, (train_ids, valid_ids) in enumerate(kf.split(UNIQ_TRAIN_IMAGE_IDS)):
    ids_valid = [UNIQ_TRAIN_IMAGE_IDS[k] for k in valid_ids]

    indices = train_df[train_df['image_id'].isin(ids_valid)].index
    train_df.loc[indices, "fold"] = fold

In [ ]:
train_df.sample(10)

In [ ]:
def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    assert image is not None, f"IMAGE NOT FOUND AT {image_path}"
    return image

In [ ]:
def get_train_transforms():
    return A.Compose(
        [
            A.RandomSizedCrop(min_max_height=(800, 800), height=1024, width=1024, p=0.5),
            A.OneOf([
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                     val_shift_limit=0.2, p=0.9),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.9),
            ],p=0.9),
            A.ToGray(p=0.01),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE, p=1),
            A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

In [ ]:
class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class WheatDataset(Dataset):
    def __init__(self, df, image_dir, transforms=None):
        super().__init__()

        self.df = df
        self.image_dir = image_dir
        self.transforms = transforms
        self.image_ids = df['image_id'].unique()

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, idx: int):
        image_id = self.image_ids[idx]
        image = load_image(os.path.join(self.image_dir, f"{image_id}.jpg")).astype(np.float32)
        image /= 255.0

        records = self.df[self.df['image_id'] == image_id]

        boxes = records[NEW_COLUMNS].values
        area = boxes[:, 2] * boxes[:, 3]
        area = torch.as_tensor(area, dtype=torch.float32)

        # change the co-ordinates into expected [x, y, x+w, y+h] format
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        # since all the boxes are wheat, it's all 1s
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor([idx])
        target["area"] = area

        if self.transforms:
            aug = self.transforms(image=image, bboxes=boxes, labels=labels)
            while len(aug['bboxes']) == 0:
                aug = self.transforms(image=image, bboxes=boxes, labels=labels)
            image = aug['image']
            target['boxes'] = torch.as_tensor(aug['bboxes'], dtype=torch.float32)

            # format yxyx
            target['boxes'][:, [0,1,2,3]] = target['boxes'][:, [1,0,3,2]]
            target['labels'] = torch.stack(aug['labels'])

        return image, target

In [ ]:
fold = 0

train_dataset = WheatDataset(
    train_df[train_df['fold'] != fold],
    os.path.join(DATA_DIR, "train"),
    transforms=get_train_transforms(),
)

validation_dataset = WheatDataset(
    train_df[train_df['fold'] == fold],
    os.path.join(DATA_DIR, "train"),
    transforms=get_valid_transforms(),
)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_data_loader = DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        sampler=RandomSampler(train_dataset),
        num_workers=1,
        collate_fn=collate_fn
    )

valid_data_loader = DataLoader(
    validation_dataset,
    batch_size=VALID_BATCH_SIZE,
    sampler=SequentialSampler(validation_dataset),
    shuffle=False,
    num_workers=1,
    collate_fn=collate_fn
)

In [ ]:
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchPredict
from effdet.efficientdet import HeadNet

def get_model():
    model_config = get_efficientdet_config('tf_efficientdet_d5')
    model = EfficientDet(model_config, pretrained_backbone=False)
    checkpoint = torch.load(os.path.join(PRETRAINED_MODELS_DIR, 'tf_efficientdet_d5-ef44aea8.pth'))
    model.load_state_dict(checkpoint)
    model_config.num_classes = 1
    model_config.image_size = IMAGE_SIZE
    model.class_net = HeadNet(model_config, num_outputs=model_config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    return DetBenchTrain(model, model_config)

In [ ]:
%%capture
model = get_model()
model.to(DEVICE)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
lr_scheduler = None

In [ ]:
def train_epoch(model, optimizer, train_data_loader, lr_scheduler):
    model.train()
    losses = AverageMeter()
    for b_idx, (images, targets) in enumerate(train_data_loader):
        images = torch.stack(images).to(DEVICE).float()

        target = {
            'bbox': [],
            'cls': []
        }

        for d in targets:
            target['bbox'].append(d['boxes'].to(DEVICE).float())
            target['cls'].append(d['labels'].to(DEVICE).float())

        output = model(images, target)
        loss = output['loss']

        losses.update(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if b_idx % 50 == 0:
            print(f"Epoch: {epoch} Batch Index: {b_idx} Loss: {loss.item()} LR: {optimizer.param_groups[0]['lr']}")
    if lr_scheduler is not None:
            lr_scheduler.step()

In [ ]:
def validate_epoch(model, valid_data_loader):
    losses = AverageMeter()
    with torch.no_grad():
        for _, (images, targets) in enumerate(valid_data_loader):
            # move the images and targets to device
            images = torch.stack(images).to(DEVICE).float()
            target = {
                'bbox': [],
                'cls': []
            }

            for d in targets:
                target['bbox'].append(d['boxes'].to(DEVICE).float())
                target['cls'].append(d['labels'].to(DEVICE).float())

            output = model(images, target)
            val_loss = output['loss']

            losses.update(val_loss.item())
    print(f"Validation Loss: {losses.avg}\n")
    return losses.avg

In [ ]:
for epoch in range(NUM_EPOCHS):
    best_val = float('inf')
    train_epoch(model, optimizer, train_data_loader, lr_scheduler)
    val_loss = validate_epoch(model, valid_data_loader)  
    if val_loss < best_val:
        best_val = val_loss
        print("SAVING MODEL")
        torch.save(model.state_dict(), os.path.join(MODELS_OUT_DIR, f"best_model.pth"))
    print(f"End of epoch {epoch}")

In [ ]:
model.eval()
torch.save(model.model.state_dict(), "test.pth")

In [ ]:
CONFIG.upload_to_kaggle("efficientdetd5", "Efficient Det D5", new=True)

In [ ]:
#INFERENCE
class WheatTestDataset(Dataset):
    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, idx: int):
        image_id = self.image_ids[idx]
        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        if self.transforms:
            image = self.transforms(image=image)['image']

        return image, image_id

In [ ]:
def get_test_transforms():
    return A.Compose([
        ToTensorV2(p=1.0)
    ])

In [ ]:
test_dataset = WheatTestDataset(test_df, os.path.join(DATA_DIR, "test"), get_test_transforms())

test_data_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

In [ ]:
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchPredict
from effdet.efficientdet import HeadNet

def load_test_model(ck_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    test_model = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size = 512
    test_model.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=0.001, momentum=0.01))

    checkpoint = torch.load(ck_path)
    test_model.load_state_dict(checkpoint)

    del checkpoint
    gc.collect()

    test_model = DetBenchPredict(test_model, config)
    test_model.eval()
    return test_model.cuda()

In [ ]:
test_model = load_test_model("test.pth")

In [ ]:
detection_threshold = 0.5
results = []

for images, image_ids in test_data_loader:
    images = torch.stack(images).to(DEVICE).float()
    with torch.no_grad():
        outputs = test_model(images, torch.tensor([1]*images.shape[0]).to(DEVICE).float(), torch.tensor([[512, 512]]*images.shape[0]).to(DEVICE).float())

        for i, image in enumerate(images):

            boxes = outputs[i]['boxes'].data.cpu().numpy()
            scores = outputs[i]['scores'].data.cpu().numpy()
            
            boxes = boxes[scores >= detection_threshold].astype(np.int32)
            scores = scores[scores >= detection_threshold]
            image_id = image_ids[i]
            
            boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
            
            result = {
                'image_id': image_id,
                'PredictionString': format_prediction_string(boxes, scores)
            }

            
            results.append(result)